In [49]:
import pandas as pd
from ripe.atlas.cousteau import (
    Dns,
    AtlasSource,
    AtlasCreateRequest
)
import datetime
from datetime import timedelta

In [50]:
FINAL_PROBES = "final_probes.csv"
ATLAS_API_KEY = "e9ab88e9-c802-4d6c-94f9-e65a9098ce14"

In [51]:
probes = pd.read_csv(FINAL_PROBES)
probes

,address_v4,address_v6,asn_v4,asn_v6,country_code,description,firmware_version,first_connected,geometry,id,is_anchor,is_public,last_connected,prefix_v4,prefix_v6,status,status_since,tags,total_uptime,type
0,87.187.99.251,2003:d8:af26:e300:ee08:6bff:fe73:2e2e,3320.0,3320.0,DE,Oglarun,5080,1.474747e+09,"{'type': 'Point', 'coordinates': [8.6505000000...",29376,False,True,1.737898e+09,87.128.0.0/10,2003::/19,"{'id': 1, 'name': 'Connected', 'since': '2025-...",1.736774e+09,"[{'name': 'Home', 'slug': 'home'}, {'name': 'N...",250485490,Probe
1,217.238.29.28,2003:d1:8f08:3220:28c1:e6ff:fee2:c356,3320.0,3320.0,DE,NaN,5080,1.713605e+09,"{'type': 'Point', 'coordinates': [7.1475, 49.4...",1008016,False,True,1.737898e+09,217.224.0.0/11,2003::/19,"{'id': 1, 'name': 'Connected', 'since': '2025-...",1.737897e+09,"[{'name': 'system: Resolves A Correctly', 'slu...",24266198,Probe
2,79.199.22.124,2003:f1:4732:5800:a62b:b0ff:fee0:1574,3320.0,3320.0,DE,t1me's pr0be,5080,1.475223e+09,"{'type': 'Point', 'coordinates': [6.9985, 49.2...",28934,False,True,1.737898e+09,79.192.0.0/10,2003::/19,"{'id': 1, 'name': 'Connected', 'since': '2025-...",1.737875e+09,"[{'name': 'VDSL', 'slug': 'vdsl'}, {'name': 's...",222453585,Probe
3,84.138.107.71,2003:e9:af05:f301:ee08:6bff:fe73:2fa6,3320.0,3320.0,DE,mirko@home-regensburg,5080,1.493236e+09,"{'type': 'Point', 'coordinates': [12.0875, 49....",30717,False,True,1.737898e+09,84.128.0.0/10,2003::/19,"{'id': 1, 'name': 'Connected', 'since': '2025-...",1.737487e+09,"[{'name': 'DSL', 'slug': 'dsl'}, {'name': 'Hom...",232307179,Probe
4,217.249.127.170,2003:73:4f40:dc00:a62b:b0ff:fee0:666,3320.0,3320.0,DE,NaN,4780,1.474651e+09,"{'type': 'Point', 'coordinates': [9.3275, 49.0...",29715,False,True,1.507368e+09,217.224.0.0/11,2003::/19,"{'id': 3, 'name': 'Abandoned', 'since': '2017-...",1.507368e+09,"[{'name': 'DSL', 'slug': 'dsl'}, {'name': 'Hom...",31437316,Probe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,81.187.219.129,2001:8b0:181:1:220:4aff:fee0:2089,20712.0,20712.0,GB,Jasons Work Probe,4790,1.330350e+09,"{'type': 'Point', 'coordinates': [-1.0715, 51....",2535,False,True,1.600967e+09,81.187.0.0/16,2001:8b0::/32,"{'id': 3, 'name': 'Abandoned', 'since': '2020-...",1.600967e+09,"[{'name': 'Office', 'slug': 'office'}, {'name'...",265520718,Probe
996,81.187.169.164,NaN,20712.0,NaN,GB,Probe-Spare2,5070,1.382475e+09,"{'type': 'Point', 'coordinates': [-0.8085, 53....",13481,False,True,1.654417e+09,81.187.0.0/16,NaN,"{'id': 3, 'name': 'Abandoned', 'since': '2022-...",1.654417e+09,"[{'name': 'NAT', 'slug': 'nat'}, {'name': 'sys...",60277413,Probe
997,81.2.125.19,NaN,20712.0,NaN,GB,meep,5080,1.398437e+09,"{'type': 'Point', 'coordinates': [-0.1395, 50....",15663,False,True,1.725627e+09,81.2.64.0/18,NaN,"{'id': 2, 'name': 'Disconnected', 'since': '20...",1.725627e+09,"[{'name': 'DSL', 'slug': 'dsl'}, {'name': 'sys...",296880666,Probe
998,217.169.19.27,2001:8b0:defe:71dc:a2f3:c1ff:fec4:645e,20712.0,20712.0,GB,NaN,5020,1.414010e+09,"{'type': 'Point', 'coordinates': [-0.0285, 51....",13085,False,True,1.616696e+09,217.169.0.0/19,2001:8b0::/32,"{'id': 3, 'name': 'Abandoned', 'since': '2021-...",1.616696e+09,"[{'name': 'Home', 'slug': 'home'}, {'name': 'N...",140675831,Probe


In [56]:
probe_ids = probes["id"].tolist()

sources = AtlasSource(
    type="probes",
    requested=len(probe_ids),
    value=",".join(map(str, probe_ids))
)

target_domains = ["google.com"]
target_resolvers = ["local", "8.8.8.8", "9.9.9.9", "1.1.1.1"]

start_time = datetime.datetime.utcnow()
stop_time = start_time + timedelta(days=1)

interval = 6 * 60 * 60  # 6 hours

# Loop through each domain and resolver
for domain in target_domains:
    for resolver in target_resolvers:
        if resolver == "local":
            measurement = Dns(
                af=4,
                query_class="IN",
                query_argument=domain,
                query_type="A",
                description=f"UDP-DNS IPv4 measurement to {domain} (local resolver)",
                use_probe_resolver=True,
                resolve_on_probe=True,
                is_oneoff=False,
                interval=interval,
            )
        else:
            measurement = Dns(
                af=4,
                query_class="IN",
                query_argument=domain,
                query_type="A",
                description=f"UDP-DNS IPv4 measurement to {domain} (resolver: {resolver})",
                use_probe_resolver=False,
                resolve_on_probe=True,
                target=resolver,
                is_oneoff=False,
                interval=interval,
            )

        # Create the request for the current measurement
        atlas_request = AtlasCreateRequest(
            start_time=start_time,
            stop_time=stop_time,
            key=ATLAS_API_KEY,  # Replace with your API key
            measurements=[measurement],
            sources=[sources],
            is_oneoff=False,
        )

        # Submit the request
        is_success, response = atlas_request.create()

        # Log the result
        if is_success:
            print(f"Measurement created successfully for {domain} with resolver {resolver}: {response}")
        else:
            print(f"Failed to create measurement for {domain} with resolver {resolver}: {response}")


/var/folders/8q/rwl87l651pv07ptk0mdv930h0000gn/T/ipykernel_5259/539290948.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.datetime.utcnow()


Failed to create measurement for google.com with resolver local: {'error': {'detail': 'There was a problem with your request', 'status': 400, 'title': 'Bad Request', 'code': 102, 'errors': [{'source': {'pointer': '/definitions/0/start_time'}, 'detail': 'Measurements must have the start time in future'}]}}
Failed to create measurement for google.com with resolver 8.8.8.8: {'error': {'detail': 'There was a problem with your request', 'status': 400, 'title': 'Bad Request', 'code': 102, 'errors': [{'source': {'pointer': ''}, 'detail': 'We do not allow more than 25 concurrent measurements to the same target: 8.8.8.8.'}]}}
Failed to create measurement for google.com with resolver 9.9.9.9: {'error': {'detail': 'There was a problem with your request', 'status': 400, 'title': 'Bad Request', 'code': 102, 'errors': [{'source': {'pointer': '/definitions/0/start_time'}, 'detail': 'Measurements must have the start time in future'}]}}
Failed to create measurement for google.com with resolver 1.1.1.1: